## Source SQLite Database

* Dataset URL: **/dsa/data/DSA-7030/cc24_7030.sqlite.db**
* Data Dictionary: [pdf](./ChicagoData-Description.pdf)
* [Chicago Crimes 2001-Present Dashboard](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present-Dashboard/5cd6-ry5g)

This SQLite database consists of a set of normalized relations populated with publically available Chicago crime data for the years 2001 to 2024.  

## Database exploration

The cells below provide SQL DML statments for examining the underlying metadata in the SQLite database that describes the table, column, and relationship details.  An initial connection and subsequent SQL statements are provided for acquiring the information necessary for reconstructing the table and relational structure in your postgres SSO database.

In [1]:
#Load extention and connect to database
%load_ext sql
%sql sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db

'Connected: @/dsa/data/DSA-7030/cc24_7030.sqlite.db'

## Explore the SQLite Tables List

This quiery simply lists the names of the database tables.  Here's a quick reference discussing the sqlite_master table showing its utility.  [sqlite_master table meta data](https://bw.org/2019/02/25/finding-meta-data-in-sqlite/) - explore what metadata is provided.

In [20]:
%%sql
SELECT distinct m.type, m.tbl_name --m.sql
FROM sqlite_master AS m,
     pragma_table_info(m.name) AS t
WHERE m.type = 'table'
order by m.name, t.pk DESC

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


type,tbl_name
table,cc24_case_location
table,cc24_cases
table,cc24_fbi_nibrs_categories
table,cc24_fbi_nibrs_offense_categories
table,cc24_iucr_codes
table,cc24_iucr_codes_primary_descriptions
table,cc24_iucr_codes_secondary_descriptions
table,cc24_nibrs_crimes_against
table,cc24_nibrs_fbicode_offenses
table,cc24_nibrs_offenses_crimes_aginst


## Explore Column Details

The query below provdes the complete list of tables and their columns with important details.

* **tbl_name** = Name of the table
* **name** = column name
* **type** = declared data type
* **notnull** = indicates column declared as NOT NULL
* **pk** = indicates column is the primary key

In [21]:
%%sql 
SELECT m.tbl_name, t.* --m.sql
 FROM pragma_table_info(m.tbl_name) t, sqlite_master m WHERE m.type='table';

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


tbl_name,cid,name,type,notnull,dflt_value,pk
cc24_iucr_codes,0,iucr_code,varchar(10),0,None,1
cc24_iucr_codes,1,iucr_index_code,char,0,None,0
cc24_iucr_codes_primary_descriptions,0,iucr_code,varchar(10),0,None,1
cc24_iucr_codes_primary_descriptions,1,iucr_primary_desc,varchar(100),0,None,0
cc24_iucr_codes_secondary_descriptions,0,iucr_code,varchar(10),0,None,1
cc24_iucr_codes_secondary_descriptions,1,iucr_secondary_desc,varchar(100),0,None,0
cc24_fbi_nibrs_categories,0,fbi_nibrs_category_name,varchar(50),0,None,1
cc24_fbi_nibrs_offense_categories,0,nibrs_offense_code,varchar(10),1,None,1
cc24_fbi_nibrs_offense_categories,1,fbi_nibrs_category_name,varchar(50),0,None,0
cc24_nibrs_crimes_against,0,nibrs_crime_against,varchar(20),1,None,1


## Below query provdes the list of columns that are declared "unique" for referential integrity enforcement.

<u>Query Output Descriptions</u>
* **name** = the table name begining at the "cc24_" -- cc24_case_location is table name.
* **unique** = indicates the column is declared "unique"
* **origin** = indicates the columns is declared as primary key
* **name_1** = column name

In [22]:
%%sql 
select il.*,ii.* --,m.sql
    from sqlite_master m, 
    pragma_index_list( m.name ) as il,
    pragma_index_info(il.name) as ii

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


seq,name,unique,origin,partial,seqno,cid,name_1
0,sqlite_autoindex_cc24_iucr_codes_1,1,pk,0,0,0,iucr_code
0,sqlite_autoindex_cc24_iucr_codes_primary_descriptions_1,1,pk,0,0,0,iucr_code
0,sqlite_autoindex_cc24_iucr_codes_secondary_descriptions_1,1,pk,0,0,0,iucr_code
0,sqlite_autoindex_cc24_fbi_nibrs_categories_1,1,pk,0,0,0,fbi_nibrs_category_name
0,sqlite_autoindex_cc24_fbi_nibrs_offense_categories_1,1,pk,0,0,0,nibrs_offense_code
0,sqlite_autoindex_cc24_nibrs_crimes_against_1,1,pk,0,0,0,nibrs_crime_against
0,sqlite_autoindex_cc24_nibrs_fbicode_offenses_1,1,pk,0,0,0,nibrs_offense_code
0,sqlite_autoindex_cc24_nibrs_offenses_crimes_aginst_1,1,pk,0,0,0,nibrs_crime_against
0,sqlite_autoindex_cc24_nibrs_offenses_crimes_aginst_1,1,pk,0,1,1,nibrs_offense_code


## Explore Relationship Details (get foreign key references)

The below query exracts the details describing the foreign key referenes bewtween tables.

* **from_table** = the name of the one-side table
* **from_column** = the name of the foreign key column in the one-side table
* **to_table** = the name of the many-side reference table
* **to_column** = the name of the foreign key column in the one-side reference table

These metadata can be translated to the necessary SQL statement to establish a relationship between tables:

```SQL
FOREIGN KEY (<from_column>) REFERENCES <to_table>(<to_column>)
```

In [5]:
%%sql
SELECT 
    m.name as from_table, f.'from' as from_column, f.'table' as to_table, f.'to' as to_column --, m.sql
FROM
    sqlite_master m
    JOIN pragma_foreign_key_list(m.name) f ON m.name != f."table"
WHERE m.type = 'table'
ORDER BY m.name
;

## Using the metadata from above:

## Implement the required CREATE TABLE statements for establishing the Chicago Crime Database in your SSO dsa_student database.  

The SQL statement takes this form:

```SQL
CREATE TABLE SSO.cc24_tbl_name (
 column_name_1 data_type <unqiue, not null>,
 column_name_N data_type <unqiue, not null>,
 PRIMARY KEY (<column_name>),
 <FOREIGN KEY (from_column_name) REFERENCES <SSO.cc24_to_table_name>(to_column_name)
 );
```

**The database tables, column names, and data types created in your SSO postgres server dsa_student database should be named exactly as they appear (with necessary modifications for any constraint or reference anomalies) in the ```cc24_7030.sqlite.db``` SQLite database.**

Use as many cells as desired.

# Connect to your SSO database using %sql magic or sqlAlchmey connection and implement your database structure (create table...)

In [2]:
#implement tables in SSO database
#######################################################################
## Code to connect and exeucte SSO.Site and SSO.Visited Tables below ## 
#######################################################################

import getpass
# This collects a masked password from the user
mypasswd = getpass.getpass()

········


In [3]:
import psycopg2
import numpy as np
from psycopg2.extensions import adapt, register_adapter, AsIs

# Then connects to the DB
connection = psycopg2.connect(database = 'dsa_student', 
                              user = 'ukgff', 
                              host = 'pgsql.dsa.lan',
                              password = mypasswd)

cursor = connection.cursor()

del mypasswd

In [30]:
#cc24_iucr_codes
sql = """
create table ukgff.cc24_iucr_codes (
    iucr_code varchar(10) unique,
    iucr_index_code char,
    primary key(iucr_code)
);
"""
cursor.execute(sql)
connection.commit()

In [31]:
#cc24_iucr_codes_secondary_descriptions
sql ="""
create table ukgff.cc24_iucr_codes_secondary_descriptions (
    iucr_code varchar(10),
    iucr_secondary_desc varchar(100),
    primary key(iucr_code),
    foreign key(iucr_code) references cc24_iucr_codes(iucr_code)
);
"""
cursor.execute(sql)
connection.commit()

In [32]:
#cc24_iucr_codes_primary_descriptions
sql ="""
create table ukgff.cc24_iucr_codes_primary_descriptions (
    iucr_code varchar(10),
    iucr_primary_desc varchar(100),
    primary key(iucr_code),
    foreign key(iucr_code) references cc24_iucr_codes(iucr_code)
);
"""
cursor.execute(sql)
connection.commit()

In [33]:
#cc24_fbi_nibrs_categories
sql = """
create table ukgff.cc24_fbi_nibrs_categories (
    fbi_nibrs_category_name varchar(50) unique,
    primary key(fbi_nibrs_category_name)
);
"""
cursor.execute(sql)
connection.commit()

In [34]:
#cc24_fbi_nibrs_offense_categories

sql = """
create table ukgff.cc24_fbi_nibrs_offense_categories (
    nibrs_offense_code varchar(10) unique not null,
    fbi_nibrs_category_name varchar(50),
    primary key(nibrs_offense_code),
    foreign key(fbi_nibrs_category_name) references cc24_fbi_nibrs_categories(fbi_nibrs_category_name)    
);
"""
cursor.execute(sql)
connection.commit()

In [66]:
#cc24_nibrs_fbicode_offenses
sql = """
create table ukgff.cc24_nibrs_fbicode_offenses (
    nibrs_offense_code varchar(10) not null,
    nibrs_offense_name varchar(100) not null,
    primary key(nibrs_offense_code),
    foreign key(nibrs_offense_code) references cc24_fbi_nibrs_offense_categories(nibrs_offense_code)
);

"""
cursor.execute(sql)
connection.commit()

In [36]:
#cc24_nibrs_crimes_against

sql = """
create table ukgff.cc24_nibrs_crimes_against (
    nibrs_crime_against varchar(20) unique not null,
    primary key(nibrs_crime_against)
);
"""
cursor.execute(sql)
connection.commit()

In [67]:
#cc24_nibrs_offenses_crimes_aginst
sql = """
create table ukgff.cc24_nibrs_offenses_crimes_aginst(
    nibrs_crime_against varchar(20),
    nibrs_offense_code varchar(10) not null,
    primary key(nibrs_crime_against,nibrs_offense_code),
    foreign key(nibrs_crime_against) references cc24_nibrs_crimes_against(nibrs_crime_against),
    foreign key(nibrs_offense_code) references cc24_nibrs_fbicode_offenses(nibrs_offense_code)
);
"""
cursor.execute(sql)
connection.commit()

In [54]:
#cc24_cases
sql = """
create table ukgff.cc24_cases (
    case_number varchar(20),
    incident_date timestamp,
    iucr_code varchar(10),
    nibrs_fbi_offense_code varchar(20),
    arrest boolean,
    domestic boolean,
    updated_on timestamp,
    primary key(case_number),
    foreign key(iucr_code) references cc24_iucr_codes(iucr_code),
    foreign key(nibrs_fbi_offense_code) references cc24_nibrs_fbicode_offenses(nibrs_offense_code)
    
);
"""
cursor.execute(sql)
connection.commit()

In [58]:
#cc24_case_location
sql = """
create table ukgff.cc24_case_location(
    case_number varchar(20),
    block varchar(100),
    location_description varchar(100),
    community_area integer,
    ward integer,
    district integer,
    beat integer,
    latitude real,
    longitude real,
    primary key(case_number),
    foreign key(case_number) references cc24_cases(case_number)
);
"""
cursor.execute(sql)
connection.commit()

In [53]:
#Drop tables 
#cursor.execute("drop table if exists ukgff.cc24_case_location; ")
#cursor.execute("drop table if exists ukgff.cc24_cases; ")
#cursor.execute("drop table if exists ukgff.cc24_nibrs_offenses_crimes_aginst; ")
#cursor.execute("drop table if exists ukgff.cc24_nibrs_crimes_against; ")
#cursor.execute("drop table if exists ukgff.cc24_nibrs_fbicode_offenses; ")
#cursor.execute("drop table if exists ukgff.cc24_fbi_nibrs_offense_categories; ")
#cursor.execute("drop table if exists ukgff.cc24_fbi_nibrs_categories; ")
#cursor.execute("drop table if exists ukgff.cc24_iucr_codes_secondary_descriptions; ")
#cursor.execute("drop table if exists ukgff.cc24_iucr_codes_primary_descriptions; ")
#cursor.execute("drop table if exists ukgff.cc24_iucr_codes; ")
#connection.commit()

## Construct and embed your Entity Relationship Diagram for your destination cc24_ postgress database

Upload your ERD image to the "final_project" folder and update the markdown below to display it here:

![ERD-HERE](M7.png)


# Perform the ETL of the source data to your SSO dsa_student Chicago Crime Database

* Establish a connection to the the SQLite source database using sqlAlchemy (best choice) - use identifiable name.
* Peform ETL of the source data tables to the destination data tables incrementally (best choice) - use identifiable name.
  * You may want to use pandas as the medium to ETL between the two databases -- **be patient!**
     * it can easliy read "big" source sql table data
     * hold data in a resizable data frame relative to computing resource constraints
     * make any necessary transformations to data values & discard duplicate records if needed
     * write/load data to destination postgresql tables
    

In [9]:
import pandas as pd
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine


postgres_db = {'drivername': 'postgres',
               'username': 'ukgff',
               'password': mypasswd,
               'host': 'pgsql.dsa.lan',
               'database' :'dsa_student'
              }
engine = create_engine(URL(**postgres_db), echo=True)

del mypasswd

In [10]:
e2 = create_engine("sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db",encoding ='utf-8')

cc24_iucr_codes
----------

In [50]:
cc24_iucr_codes  = pd.read_sql_query('select * from cc24_iucr_codes ;',e2)

In [51]:
cc24_iucr_codes.to_sql(
    'cc24_iucr_codes',
    engine,
    schema= 'ukgff',   
    if_exists='append', 
    index=False)

cc24_iucr_codes_secondary_descriptions
---------------

In [52]:
cc24_iucr_codes_secondary_descriptions = pd.read_sql_query("select* from cc24_iucr_codes_secondary_descriptions",e2)

In [53]:
cc24_iucr_codes_secondary_descriptions.to_sql(
    'cc24_iucr_codes_secondary_descriptions',
    engine,
    schema= 'ukgff',   
    if_exists='append', 
    index=False)

cc24_iucr_codes_primary_descriptions
----------

In [54]:
cc24_iucr_codes_primary_descriptions = pd.read_sql_query("select* from cc24_iucr_codes_primary_descriptions",e2)

In [55]:
cc24_iucr_codes_primary_descriptions.to_sql(
    'cc24_iucr_codes_primary_descriptions',
    engine,
    schema= 'ukgff',   
    if_exists='append', 
    index=False)

cc24_fbi_nibrs_categories
--------

In [56]:
cc24_fbi_nibrs_categories = pd.read_sql_query("select* from cc24_fbi_nibrs_categories",e2)

In [57]:
cc24_fbi_nibrs_categories.to_sql(
    'cc24_fbi_nibrs_categories',
    engine,
    schema= 'ukgff',   
    if_exists='append', 
    index=False)

cc24_fbi_nibrs_offense_categories
-------

In [59]:
cc24_fbi_nibrs_offense_categories = pd.read_sql_query("select* from cc24_fbi_nibrs_offense_categories",e2)

In [60]:
cc24_fbi_nibrs_offense_categories.to_sql(
    'cc24_fbi_nibrs_offense_categories',
    engine,
    schema= 'ukgff',   
    if_exists='append', 
    index=False)

cc24_nibrs_fbicode_offenses
------

In [61]:
cc24_nibrs_fbicode_offenses = pd.read_sql_query("select* from cc24_nibrs_fbicode_offenses",e2)

In [70]:
cc24_nibrs_fbicode_offenses.to_sql(
    'cc24_nibrs_fbicode_offenses',
    engine,
    schema= 'ukgff',   
    if_exists='append', 
    index=False)

cc24_nibrs_crimes_against
------------


In [71]:
cc24_nibrs_crimes_against = pd.read_sql_query("select* from cc24_nibrs_crimes_against",e2)

In [72]:
cc24_nibrs_crimes_against.to_sql(
    'cc24_nibrs_crimes_against',
    engine,
    schema= 'ukgff',   
    if_exists='append', 
    index=False)

cc24_nibrs_offenses_crimes_aginst
--------

In [73]:
cc24_nibrs_offenses_crimes_aginst = pd.read_sql_query("select* from cc24_nibrs_offenses_crimes_aginst",e2)

In [74]:
cc24_nibrs_offenses_crimes_aginst.to_sql(
    'cc24_nibrs_offenses_crimes_aginst',
    engine,
    schema= 'ukgff',   
    if_exists='append', 
    index=False)

cc24_cases
---

In [31]:
%%sql
select count(*) from cc24_cases;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
8202774


In [55]:
cc24_cases = pd.read_sql_query("select*from cc24_cases", e2, chunksize=500000)

for i, chunk in enumerate(cc24_cases):
    print(f"Processing chunk {i}")
    
    chunk['arrest'] = chunk['arrest'].astype(bool)
    chunk['domestic'] = chunk['domestic'].astype(bool)
    
    chunk = chunk.drop_duplicates(subset=['case_number'], keep='last')  
    
    chunk.to_sql(
        'cc24_cases',
        engine,
        'ukgff',
        if_exists='append',  
        index=False,
    )

Processing chunk 0
2024-12-06 21:35:33,258 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 21:35:33,259 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_cases'}
2024-12-06 21:35:33,731 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 21:35:46,536 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_cases (case_number, incident_date, iucr_code, nibrs_fbi_offense_code, arrest, domestic, updated_on) VALUES (%(case_number)s, %(incident_date)s, %(iucr_code)s, %(nibrs_fbi_offense_code)s, %(arrest)s, %(domestic)s, %(updated_on)s)
2024-12-06 21:35:46,538 INFO sqlalchemy.engine.base.Engine ({'case_number': 'HN549294', 'incident_date': '08/25/2007 09:22:18 AM', 'iucr_code': '0560', 'nibrs_fbi_offense_code': '08A', 'arrest': False, 'domestic': False, 'updated_on': '08/17/2015 03:03:40 PM'}, {'case_number': 'JE240540', 'incident_date': '05/

2024-12-06 21:42:40,533 INFO sqlalchemy.engine.base.Engine COMMIT
Processing chunk 3
2024-12-06 21:42:46,899 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 21:42:46,901 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_cases'}
2024-12-06 21:42:47,377 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 21:42:59,556 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_cases (case_number, incident_date, iucr_code, nibrs_fbi_offense_code, arrest, domestic, updated_on) VALUES (%(case_number)s, %(incident_date)s, %(iucr_code)s, %(nibrs_fbi_offense_code)s, %(arrest)s, %(domestic)s, %(updated_on)s)
2024-12-06 21:42:59,558 INFO sqlalchemy.engine.base.Engine ({'case_number': 'HJ404153', 'incident_date': '06/03/2003 07:14:08 AM', 'iucr_code': '0610', 'nibrs_fbi_offense_code': '05', 'arrest': False, 'domestic': False, 'updated_on': '02/28/2018

2024-12-06 21:50:15,572 INFO sqlalchemy.engine.base.Engine COMMIT
Processing chunk 6
2024-12-06 21:50:21,850 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 21:50:21,852 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_cases'}
2024-12-06 21:50:22,326 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 21:50:35,397 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_cases (case_number, incident_date, iucr_code, nibrs_fbi_offense_code, arrest, domestic, updated_on) VALUES (%(case_number)s, %(incident_date)s, %(iucr_code)s, %(nibrs_fbi_offense_code)s, %(arrest)s, %(domestic)s, %(updated_on)s)
2024-12-06 21:50:35,398 INFO sqlalchemy.engine.base.Engine ({'case_number': 'HM746514', 'incident_date': '11/28/2006 06:30:00 PM', 'iucr_code': '0610', 'nibrs_fbi_offense_code': '05', 'arrest': False, 'domestic': False, 'updated_on': '02/28/2018

2024-12-06 21:57:48,032 INFO sqlalchemy.engine.base.Engine COMMIT
Processing chunk 9
2024-12-06 21:57:54,314 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 21:57:54,315 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_cases'}
2024-12-06 21:57:54,787 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 21:58:07,020 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_cases (case_number, incident_date, iucr_code, nibrs_fbi_offense_code, arrest, domestic, updated_on) VALUES (%(case_number)s, %(incident_date)s, %(iucr_code)s, %(nibrs_fbi_offense_code)s, %(arrest)s, %(domestic)s, %(updated_on)s)
2024-12-06 21:58:07,022 INFO sqlalchemy.engine.base.Engine ({'case_number': 'HS278579', 'incident_date': '04/27/2010 01:00:00 PM', 'iucr_code': '0320', 'nibrs_fbi_offense_code': '03', 'arrest': False, 'domestic': False, 'updated_on': '02/10/2018

2024-12-06 22:06:13,127 INFO sqlalchemy.engine.base.Engine COMMIT
Processing chunk 12
2024-12-06 22:06:19,317 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 22:06:19,319 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_cases'}
2024-12-06 22:06:19,790 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 22:06:32,842 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_cases (case_number, incident_date, iucr_code, nibrs_fbi_offense_code, arrest, domestic, updated_on) VALUES (%(case_number)s, %(incident_date)s, %(iucr_code)s, %(nibrs_fbi_offense_code)s, %(arrest)s, %(domestic)s, %(updated_on)s)
2024-12-06 22:06:32,844 INFO sqlalchemy.engine.base.Engine ({'case_number': 'HX533914', 'incident_date': '12/07/2014 11:00:00 AM', 'iucr_code': '0610', 'nibrs_fbi_offense_code': '05', 'arrest': False, 'domestic': False, 'updated_on': '02/10/201

2024-12-06 22:14:49,141 INFO sqlalchemy.engine.base.Engine COMMIT
Processing chunk 15
2024-12-06 22:14:55,432 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 22:14:55,433 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_cases'}
2024-12-06 22:14:55,896 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 22:15:09,199 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_cases (case_number, incident_date, iucr_code, nibrs_fbi_offense_code, arrest, domestic, updated_on) VALUES (%(case_number)s, %(incident_date)s, %(iucr_code)s, %(nibrs_fbi_offense_code)s, %(arrest)s, %(domestic)s, %(updated_on)s)
2024-12-06 22:15:09,200 INFO sqlalchemy.engine.base.Engine ({'case_number': 'JD236008', 'incident_date': '05/13/2020 10:00:00 AM', 'iucr_code': '0820', 'nibrs_fbi_offense_code': '06', 'arrest': False, 'domestic': False, 'updated_on': '05/21/202

In [42]:
%%sql 
PRAGMA table_info(cc24_cases);

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


cid,name,type,notnull,dflt_value,pk
0,case_number,TEXT,0,None,0
1,incident_date,TEXT,0,None,0
2,iucr_code,TEXT,0,None,0
3,nibrs_fbi_offense_code,TEXT,0,None,0
4,arrest,BIGINT,0,None,0
5,domestic,BIGINT,0,None,0
6,updated_on,TEXT,0,None,0


cc24_case_location
-----

In [43]:
%%sql 
select count(*) from cc24_case_location;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
8202400


In [60]:
cc24_case_location = pd.read_sql_query("select*from cc24_case_location", e2, chunksize=500000)

for i, chunk in enumerate(cc24_case_location):
    print(f"Processing chunk {i}")
    
    chunk = chunk.drop_duplicates(subset=['case_number'], keep='last') 
    
    chunk.to_sql(
        'cc24_case_location',
        engine,
        'ukgff',
        if_exists='append',  
        index=False,
    )

Processing chunk 0
2024-12-06 22:25:59,559 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 22:25:59,560 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_case_location'}
2024-12-06 22:26:00,247 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 22:26:14,412 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_case_location (case_number, block, location_description, community_area, ward, district, beat, latitude, longitude) VALUES (%(case_number)s, %(block)s, %(location_description)s, %(community_area)s, %(ward)s, %(district)s, %(beat)s, %(latitude)s, %(longitude)s)
2024-12-06 22:26:14,413 INFO sqlalchemy.engine.base.Engine ({'case_number': 'HN549294', 'block': '074XX N ROGERS AVE', 'location_description': 'OTHER', 'community_area': 1.0, 'ward': 49.0, 'district': 24, 'beat': 2422, 'latitude': None, 'longitude': None}, {'case_number'

2024-12-06 22:34:10,292 INFO sqlalchemy.engine.base.Engine COMMIT
Processing chunk 3
2024-12-06 22:34:18,587 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 22:34:18,588 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_case_location'}
2024-12-06 22:34:19,270 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 22:34:33,199 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_case_location (case_number, block, location_description, community_area, ward, district, beat, latitude, longitude) VALUES (%(case_number)s, %(block)s, %(location_description)s, %(community_area)s, %(ward)s, %(district)s, %(beat)s, %(latitude)s, %(longitude)s)
2024-12-06 22:34:33,200 INFO sqlalchemy.engine.base.Engine ({'case_number': 'HJ350624', 'block': '018XX S KOSTNER AVE', 'location_description': 'CHA HALLWAY/STAIRWELL/ELEVATOR', 'community_area': 29.0, 'wa

2024-12-06 22:42:39,128 INFO sqlalchemy.engine.base.Engine COMMIT
Processing chunk 6
2024-12-06 22:42:47,712 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 22:42:47,713 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_case_location'}
2024-12-06 22:42:48,433 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 22:43:03,736 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_case_location (case_number, block, location_description, community_area, ward, district, beat, latitude, longitude) VALUES (%(case_number)s, %(block)s, %(location_description)s, %(community_area)s, %(ward)s, %(district)s, %(beat)s, %(latitude)s, %(longitude)s)
2024-12-06 22:43:03,737 INFO sqlalchemy.engine.base.Engine ({'case_number': 'HM769145', 'block': '008XX W ROSCOE ST', 'location_description': 'APARTMENT', 'community_area': 6.0, 'ward': 44.0, 'district': 1

2024-12-06 22:50:42,618 INFO sqlalchemy.engine.base.Engine COMMIT
Processing chunk 9
2024-12-06 22:50:50,915 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 22:50:50,916 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_case_location'}
2024-12-06 22:50:51,609 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 22:51:04,964 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_case_location (case_number, block, location_description, community_area, ward, district, beat, latitude, longitude) VALUES (%(case_number)s, %(block)s, %(location_description)s, %(community_area)s, %(ward)s, %(district)s, %(beat)s, %(latitude)s, %(longitude)s)
2024-12-06 22:51:04,966 INFO sqlalchemy.engine.base.Engine ({'case_number': 'HR691650', 'block': '051XX S ARCHER AVE', 'location_description': 'STREET', 'community_area': 57.0, 'ward': 23.0, 'district': 8,

2024-12-06 22:58:03,697 INFO sqlalchemy.engine.base.Engine COMMIT
Processing chunk 12
2024-12-06 22:58:12,115 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 22:58:12,116 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_case_location'}
2024-12-06 22:58:12,743 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 22:58:27,206 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_case_location (case_number, block, location_description, community_area, ward, district, beat, latitude, longitude) VALUES (%(case_number)s, %(block)s, %(location_description)s, %(community_area)s, %(ward)s, %(district)s, %(beat)s, %(latitude)s, %(longitude)s)
2024-12-06 22:58:27,208 INFO sqlalchemy.engine.base.Engine ({'case_number': 'HX489636', 'block': '001XX N WELLS ST', 'location_description': 'RESTAURANT', 'community_area': 32, 'ward': 42.0, 'district': 1

2024-12-06 23:06:39,718 INFO sqlalchemy.engine.base.Engine COMMIT
Processing chunk 15
2024-12-06 23:06:47,762 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2024-12-06 23:06:47,763 INFO sqlalchemy.engine.base.Engine {'schema': 'ukgff', 'name': 'cc24_case_location'}
2024-12-06 23:06:48,430 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2024-12-06 23:07:02,428 INFO sqlalchemy.engine.base.Engine INSERT INTO ukgff.cc24_case_location (case_number, block, location_description, community_area, ward, district, beat, latitude, longitude) VALUES (%(case_number)s, %(block)s, %(location_description)s, %(community_area)s, %(ward)s, %(district)s, %(beat)s, %(latitude)s, %(longitude)s)
2024-12-06 23:07:02,429 INFO sqlalchemy.engine.base.Engine ({'case_number': 'JD361167', 'block': '056XX W WILSON AVE', 'location_description': 'RESIDENCE - YARD (FRONT / BACK)', 'community_area': 15.0, 'w

In [44]:
%%sql 
PRAGMA table_info(cc24_case_location);

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


cid,name,type,notnull,dflt_value,pk
0,case_number,TEXT,0,None,0
1,block,TEXT,0,None,0
2,location_description,TEXT,0,None,0
3,community_area,BIGINT,0,None,0
4,ward,BIGINT,0,None,0
5,district,BIGINT,0,None,0
6,beat,BIGINT,0,None,0
7,latitude,TEXT,0,None,0
8,longitude,TEXT,0,None,0


# Execute SQL DML commands (using %sql magic or sqlAlchmey) to confirm the table record counts for the destination database tables are consistent with the source database table record counts

cc24_iucr_codes
------

In [45]:
%%sql
select count(*) from cc24_iucr_codes;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
437


In [46]:
cursor.execute(f"select count(*) from ukgff.cc24_iucr_codes")
result = cursor.fetchone()
result = result[0]
print(result)

437


cc24_iucr_codes_secondary_descriptions
--------------

In [47]:
%%sql
select count(*) from cc24_iucr_codes_secondary_descriptions;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
420


In [48]:
cursor.execute(f"select count(*) from ukgff.cc24_iucr_codes_secondary_descriptions")
result = cursor.fetchone()
result = result[0]
print(result)

420


cc24_iucr_codes_primary_descriptions
-----------

In [49]:
%%sql
select count(*) from cc24_iucr_codes_primary_descriptions;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
420


In [50]:
cursor.execute(f"select count(*) from ukgff.cc24_iucr_codes_primary_descriptions")
result = cursor.fetchone()
result = result[0]
print(result)

420


cc24_fbi_nibrs_categories
-------

In [51]:
%%sql
select count(*) from cc24_fbi_nibrs_categories;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
36


In [52]:
cursor.execute(f"select count(*) from ukgff.cc24_fbi_nibrs_categories")
result = cursor.fetchone()
result = result[0]
print(result)

36


cc24_fbi_nibrs_offense_categories
-----

In [53]:
%%sql
select count(*) from cc24_fbi_nibrs_offense_categories;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
91


In [54]:
cursor.execute(f"select count(*) from ukgff.cc24_fbi_nibrs_offense_categories")
result = cursor.fetchone()
result = result[0]
print(result)

91


cc24_nibrs_fbicode_offenses
-----

In [55]:
%%sql
select count(*) from cc24_nibrs_fbicode_offenses;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
91


In [56]:
cursor.execute(f"select count(*) from ukgff.cc24_nibrs_fbicode_offenses")
result = cursor.fetchone()
result = result[0]
print(result)

91


cc24_nibrs_crimes_against
----

In [57]:
%%sql
select count(*) from cc24_nibrs_crimes_against;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
4


In [58]:
cursor.execute(f"select count(*) from ukgff.cc24_nibrs_crimes_against")
result = cursor.fetchone()
result = result[0]
print(result)

4


cc24_nibrs_offenses_crimes_aginst
---

In [59]:
%%sql
select count(*) from cc24_nibrs_offenses_crimes_aginst;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
90


In [60]:
cursor.execute(f"select count(*) from ukgff.cc24_nibrs_offenses_crimes_aginst")
result = cursor.fetchone()
result = result[0]
print(result)

90


cc24_cases
---

In [56]:
%%sql
select count(*) from cc24_cases;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
8202774


In [57]:
cursor.execute(f"select count(*) from ukgff.cc24_cases")
result = cursor.fetchone()
result = result[0]
print(result)

8202370


cc24_case_location
----

In [61]:
%%sql
select count(*) from cc24_case_location;

 * sqlite:////dsa/data/DSA-7030/cc24_7030.sqlite.db
Done.


count(*)
8202400


In [62]:
cursor.execute(f"select count(*) from ukgff.cc24_case_location")
result = cursor.fetchone()
result = result[0]
print(result)

8202370
